First load the `WebCoverageService` class from the OWSLib and create a connection to a service:

In [16]:
from owslib.wcs import WebCoverageService
wcs = WebCoverageService('http://maps.isric.org/mapserv?map=/map/bdod.map', version='1.0.0')

A bounding box broadly matching Senegal:

The absense of errors means that a connection was sucessfully established. To get more information about this service, start by identifying the operations available from this service:

In [5]:
print([op.name for op in wcs.operations])

['GetCapabilities', 'DescribeCoverage', 'GetCoverage']


The full list of coverages available from this service is in the `contents` property:

In [6]:
print(list(wcs.contents))

['bdod_15-30cm_Q0.95', 'bdod_0-5cm_mean', 'bdod_5-15cm_Q0.95', 'bdod_15-30cm_Q0.5', 'bdod_30-60cm_Q0.05', 'bdod_100-200cm_Q0.05', 'bdod_15-30cm_Q0.05', 'bdod_100-200cm_Q0.95', 'bdod_0-5cm_Q0.5', 'bdod_0-5cm_Q0.05', 'bdod_5-15cm_Q0.5', 'bdod_100-200cm_mean', 'bdod_60-100cm_Q0.95', 'bdod_5-15cm_Q0.05', 'bdod_30-60cm_mean', 'bdod_5-15cm_mean', 'bdod_100-200cm_Q0.5', 'bdod_30-60cm_Q0.95', 'bdod_15-30cm_mean', 'bdod_0-5cm_Q0.95', 'bdod_60-100cm_Q0.05', 'bdod_60-100cm_Q0.5', 'bdod_60-100cm_mean', 'bdod_30-60cm_Q0.5']


That is a large set of coverages, but it is easy to filter the dictionary. For instance, to get the name of all  coverages for the 0 cm to 5 cm depth interval:

In [8]:
names = [k for k in wcs.contents.keys() if k.startswith("bdod_0-5cm")]
print(names)

['bdod_0-5cm_mean', 'bdod_0-5cm_Q0.5', 'bdod_0-5cm_Q0.05', 'bdod_0-5cm_Q0.95']


Or to search for all the coverages reporting the median prediction:

In [10]:
q0_5_covs = [k for k in wcs.contents.keys() if k.find("Q0.5") != -1]
print(q0_5_covs)

['bdod_15-30cm_Q0.5', 'bdod_0-5cm_Q0.5', 'bdod_5-15cm_Q0.5', 'bdod_100-200cm_Q0.5', 'bdod_60-100cm_Q0.5', 'bdod_30-60cm_Q0.5']


These are the SoilGrids predictions for bulk density for the six standard depths defined in the GlobalSoilMap specifications.

The details for one of these coverages can be inspected using the identifiers above:

In [14]:
bdod_5_15_median = wcs.contents['bdod_5-15cm_Q0.5']
bdod_5_15_median.supportedCRS

[urn:ogc:def:crs:EPSG::152160, urn:ogc:def:crs:EPSG::152160]

In [15]:
bdod_5_15_median.supportedFormats

['GEOTIFF_16']

In [8]:
bbox = (-17.1, 12.65, -13.15, 14.95) 

Now fetch the coverage for Senegal:

In [17]:
response = wcs.getCoverage(
    identifier='phh20_0-5cm_mean', 
    crs='urn:ogc:def:crs:EPSG::152160',
    bbox=bbox, 
    resx=1, resy=1, 
    format='GeoTIFF')

In [18]:
with open('./data/Senegal_CEC0m.tif', 'wb') as file:
    file.write(response.read())

With the data on the client side some regular interaction can be started with a library like resterIO:

In [14]:
import rasterio
CEC = rasterio.open("./data/Senegal_CEC0m.tif", driver="GTiff")
show(CEC, title='Cation Exchange Capacity at 0 metres in Senegal', cmap='gist_ncar')

RasterioIOError: './data/Senegal_CEC0m.tif' not recognized as a supported file format.

Next steps:
https://publicwiki.deltares.nl/display/OET/WCS+primer
https://geoscripting-wur.github.io/PythonRaster/